In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv("heart_clean_final.csv")

In [4]:
indep=df[['age', 'sex', 'cp', 'chol', 'thalach', 'exang', 'oldpeak', 'ca']]
dep=df["target"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.25, random_state = 0)

In [11]:
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [13]:
param_grid = {'kernel':['linear'],'gamma':['auto','scale'],'C':[0.01,0.1,1.0]} 
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1.0], 'gamma': ['auto', 'scale'],
                         'kernel': ['linear']},
             scoring='f1_weighted', verbose=3)

In [14]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [15]:
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 0.1, 'gamma': 'auto', 'kernel': 'linear'}: 0.9335130560170527


In [16]:
print("The Confusion Matrix:\n",cm)

The Confusion Matrix:
 [[ 8  2]
 [ 1 35]]


In [17]:
print("The Classification Report:\n",clf_report)

The Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.80      0.84        10
           1       0.95      0.97      0.96        36

    accuracy                           0.93        46
   macro avg       0.92      0.89      0.90        46
weighted avg       0.93      0.93      0.93        46



In [23]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
print("ROC-AU:",roc)

ROC-AU: 0.9277777777777777


In [25]:
y_train_pred = grid.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_acc)

Training Accuracy: 0.8321167883211679


In [27]:
test_acc = accuracy_score(y_test, grid_predictions)
print("Test Accuracy:", test_acc)

Test Accuracy: 0.9347826086956522


In [29]:
import pickle
filename="finalized_model_SVM.sav"

In [31]:
pickle.dump(grid,open(filename,'wb'))

In [33]:
Age=int(input("Enter your age:"))
Sex= int(input("Enter your sex (1 = male, 0 = female): "))
Cp= int(input("Enter chest pain type (0-3): "))
Chol = int(input("Enter cholesterol level (mg/dl): "))
Thalach = int(input("Enter maximum heart rate achieved: "))
Exang = int(input("Exercise induced angina (1 = yes, 0 = no): "))
Oldpeak = float(input("Enter ST depression induced by exercise: "))
Ca = int(input("Enter number of major vessels (0–3): "))

Enter your age: 28
Enter your sex (1 = male, 0 = female):  0
Enter chest pain type (0-3):  3
Enter cholesterol level (mg/dl):  230
Enter maximum heart rate achieved:  124
Exercise induced angina (1 = yes, 0 = no):  0
Enter ST depression induced by exercise:  1.5
Enter number of major vessels (0–3):  2


In [35]:
loaded_model=pickle.load(open("finalized_model_SVM.sav",'rb'))
Future_Prediction=loaded_model.predict([[Age, Sex, Cp, Chol, Thalach, Exang, Oldpeak, Ca]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
